In [8]:
# claude로 짠 코드
"""
건축법 Graph RAG - 완성 버전
목차 페이지 제외, 중복 제거, 정밀 파싱 포함
"""

import pdfplumber
import re
import os
from neo4j import GraphDatabase
from typing import List, Dict
import logging

logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


def extract_text_from_pdf(pdf_path: str, skip_toc=True) -> str:
    """
    PDF에서 텍스트 추출
    
    Args:
        skip_toc: True면 목차 페이지 자동 감지하여 제외
    """
    if not os.path.exists(pdf_path):
        return ""
    
    logger.info(f"📄 {os.path.basename(pdf_path)}")
    
    def is_toc_page(text: str) -> bool:
        """목차 페이지 판별 - 조항이 많고 본문(①)이 없으면 목차"""
        if not text:
            return True
        lines = text.split('\n')
        article_lines = [l for l in lines if re.match(r'^\s*제\d+조', l)]
        has_clauses = bool(re.search(r'[①②③④⑤]', text))
        return len(article_lines) > 15 and not has_clauses
    
    text = ""
    skipped = 0
    
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            page_text = page.extract_text()
            if not page_text:
                continue
            
            if skip_toc and is_toc_page(page_text):
                skipped += 1
                continue
            
            # 노이즈 제거
            lines = []
            for line in page_text.split('\n'):
                line = line.strip()
                if line in ["법제처", "국가법령정보센터", "건축법", "건축법 시행령", "건축법 시행규칙"]:
                    continue
                if line.isdigit() or re.match(r'^법제처\s+\d+', line):
                    continue
                lines.append(line)
            
            text += '\n'.join(lines) + "\n"
    
    logger.info(f"✅ {len(text):,}자 추출 (목차 {skipped}페이지 제외)")
    return text


class LawParser:
    """법령 파서 - 조/항/호/목 구조 파싱"""
    
    def parse(self, text: str, law_type: str) -> Dict:
        """
        법령 파싱
        """
        # 조항 분할
        articles_data = []
        
        # 정규식: 제X조(제목) - 제목 필수
        pattern = re.compile(r'\n(제\d+(?:의\d+)?조)\(([^)]+)\)')
        matches = list(pattern.finditer(text))
        
        for i, match in enumerate(matches):
            article_id = match.group(1)
            title = match.group(2)
            
            # 조항 텍스트
            start = match.start()
            end = matches[i+1].start() if i+1 < len(matches) else len(text)
            article_text = text[start:end].strip()
            
            # 항/호/목 파싱
            clauses = self._parse_clauses(article_text)
            
            # 본문이 있는 조항만 포함 (목차 제외)
            if clauses or len(article_text) > 100:
                articles_data.append({
                    'id': article_id,
                    'title': title,
                    'text': article_text,
                    'clauses': clauses
                })
        
        # 중복 제거 (같은 ID면 긴 것 우선)
        unique = {}
        for art in articles_data:
            aid = art['id']
            if aid not in unique or len(art['text']) > len(unique[aid]['text']):
                unique[aid] = art
        
        result = list(unique.values())
        logger.info(f"✅ {law_type}: {len(result)}개 조항 ({len(articles_data)-len(result)}개 중복 제거)")
        
        return {
            'law_type': law_type,
            'articles': result
        }
    
    def _parse_clauses(self, text: str) -> List[Dict]:
        """항(①②③) 파싱"""
        clauses = []
        
        # 항 마커로 분할
        parts = re.split(r'([①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳])', text)
        
        i = 0
        while i < len(parts):
            part = parts[i]
            
            # 항 마커 확인
            if re.match(r'[①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳]', part):
                # 다음 부분이 내용
                if i + 1 < len(parts):
                    clause_text = parts[i+1].strip()
                    if clause_text:
                        items = self._parse_items(clause_text)
                        clauses.append({
                            'id': part,
                            'text': clause_text,
                            'items': items
                        })
                i += 2
            else:
                i += 1
        
        return clauses
    
    def _parse_items(self, text: str) -> List[Dict]:
        """호(1. 2. 3.) 파싱"""
        items = []
        
        lines = text.split('\n')
        current_item = None
        current_lines = []
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            # 호 시작
            match = re.match(r'^(\d+)\.\s', line)
            if match:
                # 이전 호 저장
                if current_item and current_lines:
                    item_text = '\n'.join(current_lines)
                    subitems = self._parse_subitems(item_text)
                    items.append({
                        'id': current_item,
                        'text': item_text,
                        'subitems': subitems
                    })
                
                current_item = match.group(1)
                current_lines = [line]
            elif current_item:
                current_lines.append(line)
        
        # 마지막 호
        if current_item and current_lines:
            item_text = '\n'.join(current_lines)
            subitems = self._parse_subitems(item_text)
            items.append({
                'id': current_item,
                'text': item_text,
                'subitems': subitems
            })
        
        return items
    
    def _parse_subitems(self, text: str) -> List[Dict]:
        """목(가. 나. 다.) 파싱"""
        subitems = []
        
        lines = text.split('\n')
        current_sub = None
        current_lines = []
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            match = re.match(r'^([가-힣])\.\s', line)
            if match:
                if current_sub and current_lines:
                    subitems.append({
                        'id': current_sub,
                        'text': '\n'.join(current_lines)
                    })
                
                current_sub = match.group(1)
                current_lines = [line]
            elif current_sub:
                current_lines.append(line)
        
        if current_sub and current_lines:
            subitems.append({
                'id': current_sub,
                'text': '\n'.join(current_lines)
            })
        
        return subitems


class GraphBuilder:
    """Neo4j 그래프 구축"""
    
    def __init__(self, uri: str, user: str, password: str):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        logger.info(f"✅ Neo4j 연결")
    
    def close(self):
        self.driver.close()
    
    def clear(self):
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")
        logger.info("⚠️  DB 초기화")
    
    def create_indexes(self):
        """인덱스 생성"""
        with self.driver.session() as session:
            for idx in [
                "CREATE INDEX IF NOT EXISTS FOR (a:Article) ON (a.uid)",
                "CREATE INDEX IF NOT EXISTS FOR (a:Article) ON (a.article_id)",
                "CREATE INDEX IF NOT EXISTS FOR (c:Clause) ON (c.uid)",
                "CREATE INDEX IF NOT EXISTS FOR (i:Item) ON (i.uid)",
                "CREATE INDEX IF NOT EXISTS FOR (s:Subitem) ON (s.uid)",
            ]:
                try:
                    session.run(idx)
                except:
                    pass
        logger.info("✅ 인덱스")
    
    def build(self, data: Dict):
        """그래프 구축"""
        law_type = data['law_type']
        logger.info(f"🔨 {law_type} 그래프 구축...")
        
        with self.driver.session() as session:
            for art in data['articles']:
                self._build_article(session, art, law_type)
        
        logger.info(f"✅ {law_type}")
    
    def _build_article(self, session, art: Dict, law_type: str):
        """조항 및 하위 구조 생성"""
        art_uid = f"{law_type}_{art['id']}"
        
        # 조항
        session.run(f"""
            MERGE (a:Article:{law_type} {{uid: $uid}})
            SET a.article_id = $id,
                a.title = $title,
                a.law_type = $law_type,
                a.full_text = $text
        """, {'uid': art_uid, 'id': art['id'], 'title': art['title'], 
              'law_type': law_type, 'text': art['text']})
        
        # 항
        for clause in art['clauses']:
            cls_uid = f"{art_uid}_{clause['id']}"
            
            session.run("""
                MERGE (c:Clause {uid: $uid})
                SET c.clause_id = $id, c.content = $text, c.law_type = $law_type
            """, {'uid': cls_uid, 'id': clause['id'], 'text': clause['text'], 'law_type': law_type})
            
            session.run("""
                MATCH (a:Article {uid: $a})
                MATCH (c:Clause {uid: $c})      
                MERGE (a)-[:CONTAINS]->(c)
            """, {'a': art_uid, 'c': cls_uid})
            
            # 호
            for item in clause['items']:
                itm_uid = f"{cls_uid}_{item['id']}"
                
                session.run("""
                    MERGE (i:Item {uid: $uid})
                    SET i.item_id = $id, i.content = $text, i.law_type = $law_type
                """, {'uid': itm_uid, 'id': item['id'], 'text': item['text'], 'law_type': law_type})
                
                session.run("""
                    MATCH (c:Clause {uid: $c}), (i:Item {uid: $i})
                    MERGE (c)-[:CONTAINS]->(i)
                """, {'c': cls_uid, 'i': itm_uid})
                
                # 목
                for sub in item['subitems']:
                    sub_uid = f"{itm_uid}_{sub['id']}"
                    
                    session.run("""
                        MERGE (s:Subitem {uid: $uid})
                        SET s.subitem_id = $id, s.content = $text, s.law_type = $law_type
                    """, {'uid': sub_uid, 'id': sub['id'], 'text': sub['text'], 'law_type': law_type})
                    
                    session.run("""
                        MATCH (i:Item {uid: $i}), (s:Subitem {uid: $s})
                        MERGE (i)-[:CONTAINS]->(s)
                    """, {'i': itm_uid, 's': sub_uid})
    
    def create_relations(self):
        """조항 간 관계 생성"""
        logger.info("🔗 관계 생성...")
        
        with self.driver.session() as session:
            # 참조 관계
            session.run("""
                MATCH (a:Article)
                WHERE a.full_text CONTAINS '제' AND a.full_text CONTAINS '조'
                WITH a, [x IN split(a.full_text, ' ') 
                         WHERE x =~ '제\\d+(?:의\\d+)?조.*'] AS refs
                UNWIND refs AS ref
                WITH a, head(split(ref, '조')) + '조' AS ref_id
                WHERE ref_id <> a.article_id AND ref_id =~ '제\\d+(?:의\\d+)?조'
                MATCH (t:Article {law_type: a.law_type, article_id: ref_id})
                MERGE (a)-[:REFERS_TO]->(t)
            """)
            
            # 위임: 령 -> 법
            session.run("""
                MATCH (d:Decree)
                WHERE d.full_text CONTAINS '법 제'
                WITH d, [x IN split(d.full_text, ' ') WHERE x =~ '법제\\d+조.*'] AS refs
                UNWIND refs AS ref
                WITH d, '제' + substring(ref, 2, size(ref)-3) + '조' AS ref_id
                WHERE ref_id =~ '제\\d+(?:의\\d+)?조'
                MATCH (a:Act {article_id: ref_id})
                MERGE (d)-[:DELEGATES_TO]->(a)
            """)
            
            # 위임: 규칙 -> 령
            session.run("""
                MATCH (r:Rule)
                WHERE r.full_text CONTAINS '영 제'
                WITH r, [x IN split(r.full_text, ' ') WHERE x =~ '영제\\d+조.*'] AS refs
                UNWIND refs AS ref
                WITH r, '제' + substring(ref, 2, size(ref)-3) + '조' AS ref_id
                WHERE ref_id =~ '제\\d+(?:의\\d+)?조'
                MATCH (d:Decree {article_id: ref_id})
                MERGE (r)-[:DELEGATES_TO]->(d)
            """)
        
        logger.info("✅ 관계")
    
    def stats(self):
        """통계 출력"""
        with self.driver.session() as session:
            a = session.run("MATCH (a:Article) RETURN count(a)").single()[0]
            c = session.run("MATCH (c:Clause) RETURN count(c)").single()[0]
            i = session.run("MATCH (i:Item) RETURN count(i)").single()[0]
            s = session.run("MATCH (s:Subitem) RETURN count(s)").single()[0]
            cont = session.run("MATCH ()-[r:CONTAINS]->() RETURN count(r)").single()[0]
            ref = session.run("MATCH ()-[r:REFERS_TO]->() RETURN count(r)").single()[0]
            dele = session.run("MATCH ()-[r:DELEGATES_TO]->() RETURN count(r)").single()[0]
            
            print("\n" + "="*70)
            print("📊 Graph Database 통계")
            print("="*70)
            print(f"노드: Article({a:,}) + Clause({c:,}) + Item({i:,}) + Subitem({s:,}) = {a+c+i+s:,}")
            print(f"관계: CONTAINS({cont:,}) + REFERS_TO({ref:,}) + DELEGATES_TO({dele:,}) = {cont+ref+dele:,}")
            print("="*70 + "\n")


def main():
    """메인 실행"""
    
    # 설정
    NEO4J_URI = "bolt://localhost:7687"
    NEO4J_USER = "neo4j"
    NEO4J_PASSWORD = "DxI3O9BGnjGjdgu800HRd8kewNhHU9URb6lCMn3V4XI"
    
    PDFS = {
        'Act': "/home/jaehyeonpark/Downloads/건축법(법률)(제21065호)(20251001).pdf",
        'Decree': "/home/jaehyeonpark/Downloads/건축법 시행령(대통령령)(제35811호)(20251001).pdf",
        'Rule': "/home/jaehyeonpark/Downloads/건축법 시행규칙(국토교통부령)(제01531호)(20251031).pdf"
    }
    
    parser = LawParser()
    builder = GraphBuilder(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
    
    try:
        # builder.clear()  # 선택
        builder.create_indexes()
        
        for law_type, pdf_path in PDFS.items():
            logger.info(f"\n{'='*70}")
            logger.info(f"{law_type} 처리")
            logger.info(f"{'='*70}")
            
            text = extract_text_from_pdf(pdf_path, skip_toc=True)
            if text:
                parsed = parser.parse(text, law_type)
                builder.build(parsed)
        
        builder.create_relations()
        builder.stats()
        
        logger.info("\n✅ 완료!")
        
    except Exception as e:
        logger.error(f"❌ {e}", exc_info=True)
    finally:
        builder.close()


if __name__ == "__main__":
    main()

INFO - ✅ Neo4j 연결
INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE RANGE INDEX IF NOT EXISTS FOR (e:Article) ON (e.uid)` has no effect.} {description: `RANGE INDEX index_9755e5ee FOR (e:Article) ON (e.uid)` already exists.} {position: None} for query: 'CREATE INDEX IF NOT EXISTS FOR (a:Article) ON (a.uid)'
INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE RANGE INDEX IF NOT EXISTS FOR (e:Article) ON (e.article_id)` has no effect.} {description: `RANGE INDEX index_8d08c679 FOR (e:Article) ON (e.article_id)` already exists.} {position: None} for query: 'CREATE INDEX IF NOT EXISTS FOR (a:Article) ON (a.article_id)'
INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlre


📊 Graph Database 통계
노드: Article(204) + Clause(704) + Item(1,135) + Subitem(260) = 2,303
관계: CONTAINS(2,099) + REFERS_TO(782) + DELEGATES_TO(0) = 2,881

